# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from math import inf, floor
from sklearn.tree import DecisionTreeClassifier

Код для расчёта энтропии:

In [ ]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики точность (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
class DT:
	class Node:
		def __init__(self, data, left_child, right_child, feature, split_value, ig):
			self.data = data
			self.left_child = left_child
			self.right_child = right_child
			self.feature = feature
			self.split_value = split_value
			self.ig = ig

	class Leaf(Node):
		def __init__(self, data):
			super().__init__(data, None, None, None, None, None)
			categories = dict()
			for v in data.iloc[:, -1]:
				if v not in categories:
					categories[v] = 0
				else:
					categories[v] += 1
			self.category = max(categories, key=categories.get)

	def __init__(self, max_depth=inf, min_objects_in_leaf=0, max_leaves_count=inf, min_purity=0.0):
		self.max_depth = max_depth
		self.min_objects_in_leaf = min_objects_in_leaf
		self.max_leaves_count = max_leaves_count
		self.min_purity = min_purity

		self.current_depth = 0
		self.current_leaves_count = 0
		self.root = None

	def _IG(self, parent, left_child, right_child):
		H = entropy
		return H(parent) - (len(left_child) / len(parent) * H(left_child) +
							len(right_child) / len(parent) * H(right_child))

	def _split(self, data, feature, value):
		return [data[data[feature] <= value],
				data[data[feature] > value]]

	def _check_min_objects_in_leaf(self, *slices):
		for s in slices:
			if len(s.index) < self.min_objects_in_leaf:
				return False
		return True

	def _find_best_split(self, xy):
		best_ig_value = 0
		best_split_info = None
		x = xy.iloc[:, :-1]
		y = xy.iloc[:, -1]

		for feature in x:
			f = x[feature]
			for value in f:
				left_side, right_side = self._split(xy, feature, value)
				if self._check_min_objects_in_leaf(left_side, right_side):
					ig_value = self._IG(y,
										left_side.iloc[:, -1],
										right_side.iloc[:, -1])
					if ig_value > best_ig_value:
						best_ig_value = ig_value
						best_split_info = {
							'feature': feature,
							'value': value,
							'ig': best_ig_value
						}
		return best_split_info


	def _node_builder(self, xy, depth_in_node=0):
		if depth_in_node < self.max_depth and self.current_leaves_count + 1 < self.max_leaves_count:
			best_split = self._find_best_split(xy)
			if best_split is not None and best_split['ig'] > self.min_purity:
				left_child_data, right_child_data = self._split(xy, best_split['feature'], best_split['value'])
				self.current_leaves_count += 1
				return self.Node(xy,
								 self._node_builder(left_child_data, depth_in_node+1),
								 self._node_builder(right_child_data, depth_in_node+1),
								 best_split['feature'],
								 best_split['value'],
								 best_split['ig'])
		return self.Leaf(xy)

	def _predict_from(self, x, node):
		if isinstance(node, DT.Leaf):
			return node.category

		if x[node.feature] <= node.split_value:
			return self._predict_from(x, node.left_child)
		else:
			return self._predict_from(x, node.right_child)

	def fit(self, x, y):
		xy = pd.concat([x, y], axis=1)
		self.root = self._node_builder(xy)

	def predict(self, x):
		return [self._predict_from(r, self.root) for _, r in x.iterrows()]


In [ ]:
data = load_iris()
data = pd.DataFrame(data=np.c_[data['data'], data['target']], columns=data['feature_names'] + ['target'])

x_train, x_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], random_state=42)
trees = [
        DT(max_depth=2),
        DT(min_objects_in_leaf=50),
        DT(max_leaves_count=15),
        DT(min_purity=0.3),
]

res = []
for tree in trees:
	tree.fit(x_train, y_train)
	res.append(tree.predict(x_test))

for r in res:
	print("{:3.4f}".format(accuracy_score(r, y_test)))

0.9737
0.7105
0.9474
0.9737


##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
class RF:
	def __init__(self, trees_amount=5, data_samples_part=0.2, max_depth=5, min_objects_in_leaf=2, max_leaves_count=None, min_purity=0.0):
		self.trees_amount = trees_amount
		self.data_samples_part = data_samples_part
		self.data_samples_size = None
		self.trees = []

		self.max_depth = max_depth
		self.min_objects_in_leaf = min_objects_in_leaf
		self.max_leaves_count = max_leaves_count
		self.min_purity = min_purity

	def _get_samples(self, x, y):
		indexes = np.random.choice(self.data_samples_size, self.data_samples_size)
		return x.iloc[indexes], y.iloc[indexes]

	def fit(self, x, y):
		self.data_samples_size = floor(len(x.index)*self.data_samples_part)
		for i in range(self.trees_amount):
# 			print(f'{i+1}/{self.trees_amount}')
			t = DecisionTreeClassifier(max_depth=self.max_depth, 
			                           min_samples_split=self.min_objects_in_leaf, 
																 max_leaf_nodes=self.max_leaves_count, 
																 min_impurity_decrease=self.min_purity,
																 max_features="sqrt")
			x_samples, y_samples = self._get_samples(x, y)
			t.fit(x_samples, y_samples)
			self.trees.append(t)

	def predict(self, x):
		predictions = []
		for t in self.trees:
			predictions.append(t.predict(x))
		predictions = np.swapaxes(predictions, 0, 1)
		res = []
		for p in predictions:
			res.append(most_common(p))
		return res

	def get_feature_importance(self, x, y, normal_accuracy=0.85):
		res = dict()
		for feature in x:
			tmp_x = x.copy()
			tmp_x[feature] = np.random.permutation(tmp_x[feature].values)
			prediction = self.predict(tmp_x)
			res[feature] = (normal_accuracy - accuracy_score(prediction, y)) * 100
		return dict(sorted(res.items(), key=lambda item: item[1], reverse=True))


def most_common(lst):
	return np.argmax(np.bincount(lst))

In [ ]:
df_churn = pd.read_csv('churn.csv')
df_churn.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df_churn = df_churn.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
df_churn.Gender.replace(['Male', 'Female'], [1, 0], inplace=True)
df_churn.Geography.replace(['France', 'Spain', 'Germany'], [0, 1, 2], inplace=True)
df_churn.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,1,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,1,0,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
x_train_c, x_test_c, y_train_c, y_test_c = train_test_split(df_churn.iloc[:, :-1], df_churn.iloc[:, -1])

In [ ]:
f = RF(trees_amount=50, data_samples_part=0.75, max_depth=7)
f.fit(x_train_c, y_train_c)

In [ ]:
pred_c = f.predict(x_test_c)
print(accuracy_score(pred_c, y_test_c))

0.8652


In [ ]:
a = f.get_feature_importance(df_churn.iloc[:, :-1], df_churn.iloc[:, -1], accuracy_score(pred_c, y_test_c))

for z in a:
	print(z, "{:2.4f}".format(a[z]))

Age 8.4900
NumOfProducts 5.3100
IsActiveMember 2.7300
Balance 0.3100
Geography 0.2700
CreditScore -0.5200
EstimatedSalary -0.6200
Tenure -0.7000
HasCrCard -0.8000
Gender -0.8100


Данные выше показывают на какой процент изменится процент успешных распознаваний если парамерт будет случайным. Чем выше число тем параметр важнее. Параметры с оценкой близкой к нулю или отрицательной оценкой не важны и лишь вносят шум в модель.

In [ ]:
trees_amounts = [3, 5, 10, 15, 20, 50, 100]
data_samples_part = [0.01, 0.1, 0.2, 0.3, 0.5, 0.75, 0.9]
max_depth = [2, 3, 5, 7, 10, 15, 20]

all_results = []

def test_params(t_amount, d_sample, m_depth):
    r = RF(trees_amount=t_amount, data_samples_part=d_sample, max_depth=m_depth)
    r.fit(x_train_c, y_train_c)
    return accuracy_score(r.predict(x_test_c), y_test_c)

for t_amount in trees_amounts:
    print(t_amount)
    for d_sample in data_samples_part:
        for m_depth in max_depth:
            # TA - trees amount
            # DSP - the number of samples to draw from x to train each base estimator 
            # MD - max depth of trees
            all_results.append({'Accuracy': test_params(t_amount, d_sample, m_depth), 
                                'Parameters': f'TA: {t_amount} DSP: {d_sample} MD: {m_depth}'})

final_df = pd.DataFrame(all_results)

3
5
10
15
20
50
100


In [ ]:
final_df.sort_values(ascending=False, by='Accuracy')

,Accuracy,Parameters
283,0.8660,TA: 50 DSP: 0.75 MD: 7
143,0.8644,TA: 10 DSP: 0.9 MD: 7
332,0.8640,TA: 100 DSP: 0.75 MD: 7
339,0.8636,TA: 100 DSP: 0.9 MD: 7
290,0.8636,TA: 50 DSP: 0.9 MD: 7
...,...,...
3,0.7684,TA: 3 DSP: 0.01 MD: 7
50,0.7680,TA: 5 DSP: 0.01 MD: 3
1,0.7664,TA: 3 DSP: 0.01 MD: 3
54,0.7652,TA: 5 DSP: 0.01 MD: 15
